In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from icecream import ic

plt.rc('text', usetex=True)
plt.rc('font', family='STIXGeneral')

# Coincidence gamma spectroscopy
## Sodium annihilation radiation, angle at maximum signal strength

Values TOTALSUM and REALTIME taken from 'Results/4.1.4/*.mcd' files

In [2]:
ic(24/123.136)
ic(59/123.155)
ic(84/129.882)
ic(48/122.609)

ic| 24/123.136: 0.1949064449064449
ic| 59/123.155: 0.47907108927773945
ic| 84/129.882: 0.6467408878828474
ic| 48/122.609: 0.3914883899224364


0.3914883899224364

deflection| sum | time | count rate
:-   |-|-|:-
0cm  | 24 | 123.136s | 0.1949064449064449 $s^{-1}$
-1cm | 59 | 123.155s | 0.47907108927773945 $s^{-1}$
-2cm | 84 | 129.882s | 0.6467408878828474 $s^{-1}$
-3cm | 48 | 122.609s | 0.3914883899224364 $s^{-1}$

Maximal count rate at a deflection of -2cm.

## Sodium angular correlation between 
## $e^+e^- $ annihilation and $\gamma$-decay

## Cobalt decay cascade